### Import Libaries

In [ ]:
from serpapi import GoogleSearch
from datetime import date,timedelta,datetime
import numpy as np
import pandas as pd
import time
import sys
pd.set_option('display.max_columns', None) # Used for Pandas, make hidden columns/rows visiable

### Load Dataset

In [ ]:
airports = pd.read_csv('major_international_airports_clean.csv')
# temp = airports[airports['IATA_Code'].isin(['DUB','AKL'])]
# display(temp)
# airports = airports[:len(airports)]
# airports = pd.concat([airports,temp], ignore_index=True).drop_duplicates().reset_index(drop=True)
display(airports)

### Fight Data Extractor

In [ ]:
# Fligh Catalog Datset
# Airport depature & destination name, IATA airport code, date, and price, ticket sold_by   
flghDta = pd.DataFrame(columns=[
        'apt_name_dp','apt_IATA_dp','apt_time_dt_dp',
        'apt_name_ds','apt_IATA_ds','apt_time_dt_ds',
        'price','ticket_sold_by'])

def dataFightExt(data): # Process data for 
    for fd in data:
        fligths = fd['flights'][0]
      
        dp,ds= fligths['departure_airport'],fligths['arrival_airport']
        dp_name,ds_name = dp['name'],ds['name']
        dp_IATA,ds_IATA = dp['id'],ds['id'] 

        dp_dttm,ds_dttm = dp['time'],ds['time']
        dp_dttm_obj = datetime.strptime(dp_dttm,'%Y-%m-%d %H:%M')
        ds_dttm_obj = datetime.strptime(ds_dttm,'%Y-%m-%d %H:%M')
        dp_dt = dp_dttm_obj.date()
        ds_dt = ds_dttm_obj.date()

        try:
            tck_sld_by = fligths['ticket_also_sold_by']
        except:
            tck_sld_by = []

        price = fd['price']

        flghDta.loc[len(flghDta)] = [
            dp_name,dp_IATA,dp_dt,
            ds_name,ds_IATA,ds_dt,
            price,tck_sld_by]

### SerpAPI for Future Flight Data

In [ ]:
i = -1 # When issuess happen set this to the last value/completed here to resume after fixing said issuess with code to mange unexpected errors
outbound = date(2025,12,11) # Flight time range
EndPoint = date(2026,6,11)

des = ['DUB','AKL'] # Our destinations

Limit = False #Used for monthly usage control 
for idx,d in airports.iterrows(): # Take a Airport user would dep from and find flight paths to des, then save to Catalog
  dep = d['IATA_Code'] 
  if idx < i: continue # Used with resume operations after hitting call limit or when internet issues happen

  hold = '' # We dont want dep and des to be the same DUB and [DUB,AKL] you want DUB and [AKL] 
  if dep in des: hold = des.pop(des.index(dep))
  
  while outbound <= EndPoint: # go through every 2 days grabing flight paths
    print(f"{idx} || {outbound}//{EndPoint} || DEP: {dep} || DES: {des}", end='\r') # This helps with knowing what flight we are on

    params = {
      "api_key": "API_KEY", 
      "engine": "google_flights",
      "hl": "en",
      "gl": "ca",
      "departure_id": dep,
      "arrival_id": ','.join(des),
      "outbound_date": str(outbound),
      "currency": "CAD",
      "sort_by": "3",
      "type": "2",
      "stops": "1"
    }

    chkact = GoogleSearch(params).get_account()
    if chkact['this_month_usage'] > 4890: # Limiter
      Limit = True
      break
  
    search = GoogleSearch(params)
    results = search.get_dict()

    try:
      dataFightExt(results['other_flights'])
    except Exception as e: # Happens when other_flights doesn't exist
      pass

    outbound += timedelta(days=2) # move to next time point
    
    sys.stdout.flush()
    time.sleep(0.1)

  # print(f"")

  if Limit == True: break

  if hold != '': des.append(dep) # refill the des list 
  
  outbound = date(2025,11,29) # reset start date for the next dep
  
flghDta = flghDta.convert_dtypes()
display(flghDta.head(5))

In [ ]:
flghDta.info()

In [ ]:
flghDta2 = flghDta.convert_dtypes()
display(flghDta2.head(5))

In [ ]:
flghDta2.info()

In [ ]:
# Inserting the Country and City columns
flghDta2.insert(4,'City_dp',None)
flghDta2.insert(4,'Country_dp',None)
flghDta2.insert(10,'City_ds',None)
flghDta2.insert(11,'Country_ds',None)
flghDta2.head(3)

In [ ]:
# Addng the Country and City from the major_international_airport dataset for each flight path 
flghDtacpy = flghDta2.copy()
for idx,d in airports.iterrows():
    code = d['IATA_Code']
    Country = d['Country']
    City = d['City']
    flghDtacpy.loc[flghDta2['apt_IATA_dp'] == code,'City_dp'] = City
    flghDtacpy.loc[flghDta2['apt_IATA_dp'] == code,'Country_dp'] = Country
    flghDtacpy.loc[flghDta2['apt_IATA_ds'] == code,'City_ds'] = City
    flghDtacpy.loc[flghDta2['apt_IATA_ds'] == code,'Country_ds'] = Country
flghDtacpy.isnull().any()

In [ ]:
flghDtacpy.head(10)

In [ ]:
output_path = "flight_paths.csv"
flghDtacpy.to_csv(output_path, index=False)

In [ ]:
flghDtacpy.shape